In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import statsmodels.api as sm
import scipy
import time

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
Normal = tfd.MultivariateNormalFullCovariance
slim=tf.contrib.slim
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import argparse
import multiprocessing as mp

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--grad', '-g', type=str, default='u2g', \
                    help='reinforce, arm, u2g')
parser.add_argument('--lmbd',  type=float, default=10., help='lmbd')
parser.add_argument('--lr', '-l', type=float, default=0.01, help='lr')

parser.add_argument('--name', '-n', default='exp2_vb', help='exp,model name')
parser.add_argument('--sigma',  type=float, default=1.414, \
                    help='variance of epsilon, 1.195 or 1.414 for SNR=7 or 5')
parser.add_argument('--sigma_beta',  type=float, default=1.0, \
                    help='variance of slab prior')
parser.add_argument('--N', type=int, default=100, help='number of data')
parser.add_argument('--P', '-b', type=int, default=1000, help='dimension size')
parser.add_argument('--K', type=int, default=10, help='number of MC sample')

parser.add_argument('--iter',  type=int, default=10000, help='iter')
parser.add_argument('--N_test', type=int, default=10, \
                    help='number of repeated experiment')

#args = parser.parse_args()
args, unknown = parser.parse_known_args()  
eps = 1e-8

In [3]:
def bernoulli_loglikelihood(b, log_alpha):
    return b * (-tf.nn.softplus(-log_alpha)) + (1 - b) * \
                        (-log_alpha - tf.nn.softplus(-log_alpha))

def log_p_yz(E, x, y):
    def ff(E, x, y, sigma, sigma_beta):
        '''
        compute covariance matrix when active set is not empty
        '''
        idx = tf.squeeze(tf.where(tf.not_equal(E,0)))       
        X1 = tf.gather(x,idx,axis=1)
        X1 = tf.reshape(X1,[args.N,-1])
        V0 = tf.matmul(X1, tf.transpose(X1))
        cov = (sigma_beta**2)*V0 + (sigma**2) * tf.eye(args.N)
        return cov
    cov = tf.cond(tf.greater(tf.reduce_sum(E), 1e-2), lambda:ff(E, x, y,  \
                  args.sigma, args.sigma_beta),\
                  lambda:(args.sigma**2) * tf.eye(args.N))
    mean = tf.zeros(args.N)
    mvn = Normal(mean, cov)
    return mvn.log_prob(y)


def data_gen(N, P, sigma,seed=1):
    beta_true = np.array([1.]*10+[0.]*(P-10))  
    def cov(P, rho = 0.):
        V = np.empty([P,P])
        for i in range(P):
            for j in range(i):
                V[i,j] = np.power(rho, np.abs(i-j))
                V[j,i] = V[i,j]
        for i in range(P):
            V[i,i] = 1    
        return V
    V = cov(P)
    M = np.array([0]*P)
    ############
    np.random.seed(seed)
    ############
    X = np.random.multivariate_normal(mean=M, cov=V, size = N)
    noise = np.random.normal(scale = sigma, size = N)
    Y = np.matmul(X, beta_true) + noise
    Y = Y.astype(np.float32) 
    X = X.astype(np.float32) 
    return X, Y, V, beta_true

def main(ss): 
    X,Y,COV,beta_true = data_gen(args.N,args.P,args.sigma,seed=ss)    
    z_true = (beta_true>0)
    SNR = np.matmul(np.matmul(beta_true, COV),beta_true) / (args.sigma**2)  

    tf.reset_default_graph(); 
     
    x = tf.placeholder(tf.float32,[args.N, args.P],name='data_x')  
    y = tf.placeholder(tf.float32,[args.N],name='data_y')   
    
    phi = tf.get_variable("phi", dtype=tf.float32, \
                          initializer=tf.random.normal([args.P]))
    prob = tf.sigmoid(phi)
    
    def fun(E):
        logit_pi = - args.lmbd/(2*args.sigma**2)    
        log_p_y_given_z = log_p_yz(E, x, y)
        log_p_z = tf.reduce_sum(bernoulli_loglikelihood(E, logit_pi))
        log_q_z = tf.reduce_sum(bernoulli_loglikelihood(E, phi))
        return log_p_z+log_p_y_given_z-log_q_z
        
    u_noise = tf.random_uniform(shape=[args.K,args.P],maxval=1.0)  #K*P
    E1 = tf.cast(u_noise>tf.sigmoid(-phi),tf.float32)
    E2 = tf.cast(u_noise<tf.sigmoid(phi),tf.float32)   
    if args.grad == 'arm' or 'u2g':
        Fun1 = tf.map_fn(fun, E1)       
        Fun2 = tf.map_fn(fun, E2)
        F_term = Fun1 - Fun2
        elbo = tf.reduce_mean(Fun2)
    elif args.grad == 'reinforce' :
        F_term = tf.map_fn(fun, E2)
        elbo = tf.reduce_mean(Fun2)
    else:
        raise ValueError('No grad defined')
    if len(np.shape(F_term))<2:
        F_term = F_term[:,None]  #K*1
    
    if args.grad == 'arm':
        G = F_term*(u_noise - 0.5)
        phi_tile = tf.tile(phi[None,:], [args.K,1]) #K*P  
        mask = tf.to_float(tf.abs(E1 - E2))
        G_mask = G * mask
        grad = tf.reduce_mean(G_mask,axis=0) 
    elif args.grad == 'u2g':
        phi_tile = tf.tile(phi[None,:], [args.K,1])
        current_prob = tf.sigmoid(phi_tile)
        G = F_term * tf.to_float(E1 - E2) * \
                        tf.maximum(current_prob,1-current_prob)/2.0
        grad = tf.reduce_mean(G,axis=0) 
    elif args.grad == 'reinforce':
        phi_tile = tf.tile(phi[None,:], [args.K,1]) 
        current_prob = tf.sigmoid(phi_tile)
        G = F_term * (tf.to_float(E2)-current_prob)
        grad = tf.reduce_mean(G,axis=0) 
                    
    train_opt = tf.train.GradientDescentOptimizer(args.lr)
    
    gradvars = zip([-grad], [phi])
    train_op = train_opt.apply_gradients(gradvars)
  
    init_op=tf.global_variables_initializer()
    
    sess=tf.InteractiveSession()
    sess.run(init_op)    
    cost_r = []; entropy_r=[]; 
    
    start = time.time()
    for i in range(args.iter):   
        _,cost = sess.run([train_op, elbo],{x:X, y:Y})
        cost_r.append(cost)  
        probz = np.squeeze(sess.run(prob))           
        entropy = np.sort(- probz * np.log(probz + eps))
        entropy_r.append(np.mean(entropy))
        if np.mean(entropy[-args.P//100:])<0.2:
            break      
    
    probability = np.squeeze(sess.run(prob))
    z_hat = np.array(probability>0.5).astype(np.int32)  
    beta_hat = np.zeros([args.P])
    if sum(np.abs(z_hat))>eps:
        X1 = np.take(X, np.squeeze(np.where(z_hat != 0)), axis=1)
        if sum(np.abs(z_hat)) == 1:
            X1 = X1[:,None]
        beta_hat0 = scipy.linalg.lstsq(X1, Y,cond=eps, lapack_driver= 'gelsy')[0]
        beta_hat[np.where(z_hat != 0)] = beta_hat0    
    
    #Evaluation metric
    ols_results = sm.OLS(Y, X).fit()
    beta_ols = ols_results.params
    
    TP = np.sum(z_true * z_hat ) 
    FP = np.sum((1-z_true) * z_hat) 
    FN = np.sum(z_true * (1-z_hat)) 
    
    prec = TP/(TP + FP + + eps)   
    rec = TP/(TP + FN + eps)
    F1 = 2 * prec * rec / (prec + rec + eps)
         
    RTE_ols = 1 + np.matmul(np.matmul((beta_true-beta_ols), COV),\
                            (beta_true-beta_ols)) / (args.sigma**2)    
    RTE_grad = 1 + np.squeeze(np.matmul(np.matmul((beta_hat-beta_true), COV),\
                             (beta_hat-beta_true).T) / (args.sigma**2))
    SNR = np.matmul(np.matmul(beta_true, COV),beta_true) / (args.sigma**2)       
    

    print('Trial-'+str(ss),': prec =', round(prec,2), 'rec =', round(rec,2), \
           'Iter =', i)    
    
    corr = np.sum((1-z_true) * (1-z_hat))
    incorr = FN
    duration = time.time()-start
    all_ = [prec, rec, F1, corr, incorr, RTE_ols, \
            RTE_grad, SNR, duration, args.lmbd]
    sess.close()
    return(all_)

In [4]:
if __name__ == '__main__':
    start = time.time()
    pool = mp.Pool(mp.cpu_count()-1)
    results = pool.map(main, [i for i in range(args.N_test)])
    print('Time per trial is',(time.time()-start)/args.N_test)

Trial-1 : prec = 1.0 rec = 1.0 Iter = 4868
Trial-3 : prec = 0.75 rec = 0.9 Iter = 9999
Trial-0 : prec = 1.0 rec = 1.0 Iter = 9999
Trial-4 : prec = 0.67 rec = 0.6 Iter = 9999
Trial-2 : prec = 1.0 rec = 1.0 Iter = 9999
Trial-6 : prec = 1.0 rec = 1.0 Iter = 9999
Trial-5 : prec = 1.0 rec = 1.0 Iter = 9999
Trial-7 : prec = 0.91 rec = 1.0 Iter = 8925
Trial-9 : prec = 1.0 rec = 1.0 Iter = 9999
Trial-8 : prec = 1.0 rec = 1.0 Iter = 9999
Time per trial is 22.6846678019
